# LLM Symbolic Math 
让 LLM 把自然语言的数学问题转成 SymPy 的数值表达式，然后计算。其底层使用了 [SymPy](https://www.sympy.org/en/index.html).

## LLMSymbolicMathChain 参数 allow_dangerous_requests
- allow_dangerous_requests=False
只允许一部分 白名单运算（加减乘除、幂运算、基本函数等）。安全，不会执行任意代码。

- allow_dangerous_requests=True
允许 LLM 构造任意 SymPy 表达式，可能导致安全风险（比如执行恶意代码）。不推荐在生产环境使用。

In [12]:
from langchain_experimental.llm_symbolic_math.base import LLMSymbolicMathChain
from langchain_openai import OpenAI

import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()

# 使用 通义千问 代替openai
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",   # ✅ 用通义千问提供的模型名，比如 qwen-plus / qwen-max / qwen-turbo
    temperature=0
)
llm_symbolic_math = LLMSymbolicMathChain.from_llm(
    llm, 
    allow_dangerous_requests=True
)

## Integrals and derivates

In [13]:
llm_symbolic_math.invoke("What is the derivative of sin(x)*exp(x) with respect to x?")

{'question': 'What is the derivative of sin(x)*exp(x) with respect to x?',
 'answer': 'Answer: exp(x)*sin(x) + exp(x)*cos(x)'}

In [14]:
llm_symbolic_math.invoke(
    "What is the integral of exp(x)*sin(x) + exp(x)*cos(x) with respect to x?"
)

{'question': 'What is the integral of exp(x)*sin(x) + exp(x)*cos(x) with respect to x?',
 'answer': 'Answer: exp(x)*sin(x)'}

## Solve linear and differential equations

In [15]:
llm_symbolic_math.invoke('Solve the differential equation y" - y = e^t')

{'question': 'Solve the differential equation y" - y = e^t',
 'answer': 'Answer: Eq(y(t), C1*exp(-t) + C2*exp(t))'}

In [16]:
llm_symbolic_math.invoke("What are the solutions to this equation y^3 + 1/3y?")

{'question': 'What are the solutions to this equation y^3 + 1/3y?',
 'answer': 'Answer: {0, -sqrt(3)*I/3, sqrt(3)*I/3}'}

In [19]:
llm_symbolic_math.invoke("x = y + 5, y = z - 3, z = x * y. Solve for x, y, z")

ValueError: LLMSymbolicMathChain._evaluate("
solveset((x - (y + 5), y - (z - 3), z - (x * y)), (x, y, z))
") raised error: (x - y - 5, y - z + 3, -x*y + z) is not a valid SymPy expression. Please try again with a valid numerical expression